In [ ]:
import sys
sys.path.append("./drive_interfaces/carla/carla_client/")
from carla.client import make_carla_client
from carla.settings import CarlaSettings
from carla.client import VehicleControl

In [ ]:
with open("./drive_interfaces/carla/yang_template.ini", "r") as f:
    print(f.read())

In [ ]:
import random
from carla.sensor import Camera, Lidar
settings = CarlaSettings()
settings.set(
    SynchronousMode=True,
    SendNonPlayerAgentsInfo=True,
    NumberOfVehicles=20,
    NumberOfPedestrians=40,
    WeatherId=random.choice([1, 3, 7, 8, 14]),
    QualityLevel="Low")
settings.randomize_seeds()

# Now we want to add a couple of cameras to the player vehicle.
# We will collect the images produced by these cameras every
# frame.

# The default camera captures RGB images of the scene.
camera0 = Camera('CameraRGB')
# Set image resolution in pixels.
camera0.set_image_size(800, 600)
# Set its position relative to the car in meters.
camera0.set_position(0.30, 0, 1.30)
settings.add_sensor(camera0)

# Let's add another camera producing ground-truth depth.
camera1 = Camera('CameraDepth', PostProcessing='Depth')
camera1.set_image_size(800, 600)
camera1.set_position(0.30, 0, 1.30)
settings.add_sensor(camera1)

print(str(settings))

In [ ]:
from carla.client import CarlaClient
client = CarlaClient("127.0.0.1", 2000)
client.connect()
with open("./drive_interfaces/carla/yang_template.ini", "r") as f:
    #client.load_settings(settings)
    client.load_settings(f.read())
client.start_episode(0)
measurements, sensor_data = client.read_data()
client.send_control(VehicleControl())

client.disconnect()

In [ ]:
with make_carla_client("127.0.0.1", 2000) as client:
    with open("./drive_interfaces/carla/yang_template.ini", "r") as f:
        #client.load_settings(settings)
        client.load_settings(f.read())
    client.start_episode(0)
    measurements, sensor_data = client.read_data()
    client.send_control(VehicleControl())

In [ ]:
measurements.player_measurements

In [ ]:
sensor_data.keys()

In [ ]:
image_converter.depth_to_array(sensor_data['DepthLeft']).shape

In [ ]:
a=image_converter.labels_to_array(sensor_data['SegLeft'])

In [ ]:
import numpy as np
b=a[:,:,np.newaxis]

In [ ]:
b.shape

In [ ]:
import scipy.misc
image = scipy.misc.imresize(b, [30, 40], interp='nearest')

In [ ]:
a=sensor_data['DepthLeft']

In [ ]:
from carla import image_converter

In [ ]:
b=image_converter.depth_to_array(a)
b.shape

In [ ]:
b

In [ ]:
measurements

In [ ]:
measure, sensor =

In [ ]:
path= "/scratch/yang/aws_data/carla_collect/1/2018726_default_RotationPitch=-15_WeatherId=10_4/data_00000.h5"
import h5py
f=h5py.File(path, "r")

In [ ]:
f.keys()

In [ ]:
import cv2
image = cv2.imdecode(f['CameraLeft'][0], 1)

In [ ]:
from PIL import Image
a=Image.fromarray(image)

In [ ]:
a

In [ ]:
path= "/scratch/yang/aws_data/carla_collect/1/2018726_default_RotationPitch=-15_WeatherId=10_5/data_00000.h5"
path = "/data/yang/code/aws/scratch/carla_collect/1/2018727_default_RotationPitch=0_WeatherId=13_1/data_00000.h5"
path = "/scratch/yang/aws_data/carla_collect/1/2018730_default_RotationPitch=0_WeatherId=01_1/data_00000.h5"
def sample_images_from_h5(path):
    f=h5py.File(path, "r")
    for key in f.keys():
        if 'targets' not in key:
            for imid in range(200):
                data = f[key][imid]
                if 'camera' in key.lower():
                    ext = ".jpg"
                else:
                    ext = ".png"
                out_name = path+"." + key + str(imid).zfill(4) + ext
                with open(out_name, "w") as g:
                    g.write(data)
sample_images_from_h5(path)

"ffmpeg -i data_00000.h5.CameraMiddle%04d.png -c:v libx264 out.mp4"

In [ ]:
from subprocess import call
for i in range(1):
    prefix = "/scratch/yang/aws_data/carla_collect/1/2018730_default_RotationPitch=0_WeatherId="+str(i).zfill(2)+"_1/"
    path = prefix + "data_00000.h5"
    ! ls -alh $path
    sample_images_from_h5(path)
    
    call("ffmpeg -i "+ prefix +"data_00000.h5.CameraMiddle%04d.jpg -c:v libx264 "+ prefix[:-1]+".mp4", shell=True)

In [ ]:
import threading
from multiprocessing import Process
import time

class ControlInterface(object):
    def __init__(self):
        self._throttle = 1.0       
        self.counter = 0        
        self.start_loop()
        self.last_time = time.time()

    def set_throttle(self, new_throttle):
        self._throttle = new_throttle

    def start_loop(self):
        self.counter += 1
        if self.counter % 75 == 0:
            print("time eplapsed is ", time.time()-self.last_time)
            self.last_time = time.time()
            print( "throttle", self._throttle)
        threading.Timer(1.0/75, self.start_loop).start()
        

ctrl = ControlInterface()
print("after constructor")
ctrl.set_throttle(-1.0)

In [ ]:
import os
def get_files_count_in_folder(folder):
    if not os.path.exists(folder):
        return 0
    else:
        pass
import glob
len(glob.glob("/data/yang/code/aws/scratch/carla_collect/1/2018731_default_RotationPitch=0_WeatherId=01_1/*.h5"))

In [ ]:
import os, threading
cmd = ['bash', '-c', " '/scratch/yang/aws_data/carla_0.8.4/CarlaUE4.sh  -carla-server -carla-settings=/data/yang/code/aws/CIL_modular/drive_interfaces/carla/yang_template.ini -benchmark -fps=5' "]
#cmd = cmd.split(" ")
#cmd = [x for x in cmd if x!="" ]
print(" ".join(cmd))
print("before spawnling")
t = threading.Thread(target = lambda : os.system(" ".join(cmd)))
t.start()       
#! ps aux | grep $id0

In [ ]:
len(glob.glob("/data/yang/code/aws/scratch/carla_collect/1/*/data_*.h5"))

In [ ]:
path = "/scratch/yang/aws_data/CIL_modular_data/matthias_data/RC025Val/data_00000.h5"
import h5py

In [ ]:
f=h5py.File(path, "r")

a=f['rgb']

In [ ]:
from PIL import Image
i+=3
print(i)
Image.fromarray(a[i, :,:,:])

In [ ]:
path ="/scratch/yang/aws_data/carla_collect/1/default_ImageSizeX=700_WeatherId=10/data_00099.h5"
import h5py, time
f = h5py.File(path, "r")

In [ ]:
import cv2
start = time.time()
t_decode = 0.0
t_resize = 0.0
for i in range(200):
    a= f["CameraMiddle"][i]
    start = time.time()
    decoded = cv2.imdecode(a, 1)                    
    t_decode += time.time()-start
    sz = (88,200)
    start = time.time()
    decoded = cv2.resize(decoded, (sz[1], sz[0]))
    t_resize += time.time() -start
delta = time.time()-start
print(delta, 200/delta)
print(t_decode, t_resize)

In [ ]:
91*20*200 /1000/1000.0

In [ ]:
8*75/1.5

In [ ]:
from joblib import Parallel, delayed
p = Parallel(n_jobs=8, backend="threading", verbose=10)


func = lambda x : cv2.imdecode(x, 1)
if True:
    height, width = 88, 200
    func_previous = func
    func = lambda x : cv2.resize(func_previous(x), (width, height))
                
func = lambda x : cv2.resize(cv2.imdecode(x, 1), (200, 88))        
func = delayed(func)
results = p(func(f["CameraMiddle"][0]) for i in range(8*75))

In [ ]:
results = p(delayed(cv2.imdecode)(f["CameraMiddle"][i], 1) for i in range(200))

In [ ]:
from PIL import Image
Image.fromarray(results[0])

In [ ]:
import numpy as np
a = np.concatenate((results[0], results[0]+5), axis=2)

In [ ]:
b=cv2.resize(a, (200, 300))


In [ ]:
b.shape

In [ ]:
Image.fromarray(b[:,:,:3])

In [ ]:

def _resize_BHWC(input, sz):
    B, H, W, C = input.shape
    input = np.transpose(input, (1, 2, 3, 0))
    # now input has size H W C B
    input = np.reshape(input, (H, W, C*B))
    print(input.shape)
    output = cv2.resize(input[:,:,:1000], (sz[1], sz[0]))
    output = np.reshape(output, (sz[0], sz[1], C, B))
    output = np.transpose(output, (3, 0, 1, 2))
    return output


In [ ]:
stk = np.stack(results, axis = 0)

In [ ]:
stk.shape

In [ ]:
out = _resize_BHWC(stk, (200, 300))

In [ ]:
func = lambda x: cv2.imdecode(x, 1)
if hasattr(self._config, "hack_resize_image"):
    height, width = self._config.hack_resize_image
    func_previous = func
    func = lambda x: cv2.resize(func_previous(x), (width, height))


# measure the segmentation difference

In [ ]:
path = "/scratch/yang/aws_data/carla_collect/10/default_PositionZ=1.4_WeatherId=03/data_00000.h5"
import h5py
f = h5py.File(path, "r")


In [ ]:
import cv2
for i in range(200):
    image = f['CameraMiddle'][i]
    image = cv2.imdecode(image, 1)
    image = image[:,:, ::-1]
    seg = f['SegMiddle'][i]
    seg = cv2.imdecode(seg, 0)
    break

In [ ]:
seg.shape

0	None
1	Buildings
2	Fences
3	Other
4	Pedestrians
5	Poles
6	RoadLines
7	Roads
8	Sidewalks
9	Vegetation
10	Vehicles
11	Walls
12	TrafficSigns

In [ ]:
# 0          1            2           3               4               5      6
{'Ignored', 'Movable', 'Navigable', 'NoneNavigable', 'StaticLayout', 'Sky', 'Lane'}

In [ ]:
from PIL import Image
Image.fromarray(image)

In [ ]:
import numpy as np
mapping = np.array([0, 4, 3, 0, 1, 4, 6, 2, 3, 4, 1, 4, 4])
seg_cvt = mapping[seg]

In [ ]:
import sys
sys.path.append("/data/yang/code/aws")
from all_perceptions import Perceptions

In [ ]:
p = Perceptions(det_COCO=False,
                 det_TL=False,
                 det_TS=False,
                 seg=True,
                 depth=False,
                 batch_size=1, # batch_size could also be a dict
                 gpu_assignment=[0],
                 compute_methods={},
                 viz_methods={},
                 num_replicates={},
                 path_config="path_jormungandr")

In [ ]:
image_batch = np.expand_dims(image, 0)
out = p.compute(image_batch)

In [ ]:
out = np.squeeze(out['seg'])

In [ ]:
out = np.argmax(out, axis=2) + 1 # 0 does not exists, since it is the ignore class underhood

In [ ]:
out = cv2.resize(out, (out.shape[1]*4, out.shape[0]*4), interpolation=cv2.INTER_NEAREST)

In [ ]:
out.shape

In [ ]:
#sky is ignored, also the ignore class

In [ ]:
Image.fromarray((out==1).astype(np.uint8)*255)

In [ ]:
Image.fromarray((seg_cvt==1).astype(np.uint8)*255)

In [ ]:
def compute_inter_union(gt, pred, nclasses):
    # return a nclass * 2 matrix: [iclass][inter, union]
    out = np.zeros((nclasses, 2), dtype=np.int64)
    pred = pred[gt!=0] - 1 
    gt = gt[gt!=0] - 1
    for i in range(nclasses):
        out[i, 0] = np.sum(np.logical_and(pred == i, gt == i))
        out[i, 1] = np.sum(np.logical_or (pred == i, gt == i))
    return out

In [ ]:
compute_inter_union(seg_cvt, out, 6)

In [ ]:
import pickle
with open("../seg.pkl", "r") as f:
    res = pickle.load(f)

In [ ]:
res

In [ ]:
perf.focal

In [ ]:
import h5py, cv2, sys, pickle, math, numpy
import numpy as np
sys.path.append("/data/yang/code/aws")
from all_perceptions import Perceptions

debug = None

class DepthPerformance():
    def __init__(self):
        width = 768
        fov = 90.0
        self.focal = width / (2.0 * math.tan(fov * math.pi / 360.0))
        self.baseline = 0.54

        self.p = Perceptions(det_COCO=False,
                             det_TL=False,
                             det_TS=False,
                             seg=False,
                             depth=True,
                             batch_size=1,  # batch_size could also be a dict
                             gpu_assignment=[0],
                             compute_methods={},
                             viz_methods={},
                             num_replicates={},
                             path_config="path_jormungandr")

        self.reset()

    def _compute_errors(self, gt, pred):
        thresh = np.maximum((gt / pred), (pred / gt))
        a1 = (thresh < 1.25).mean()
        a2 = (thresh < 1.25 ** 2).mean()
        a3 = (thresh < 1.25 ** 3).mean()

        rmse = (gt - pred) ** 2
        rmse = np.sqrt(rmse.mean())

        rmse_log = (np.log(gt) - np.log(pred)) ** 2
        rmse_log = np.sqrt(rmse_log.mean())

        abs_rel = np.mean(np.abs(gt - pred) / gt)

        sq_rel = np.mean(((gt - pred) ** 2) / gt)

        return abs_rel, sq_rel, rmse, rmse_log, a1, a2, a3

    def reset(self):
        self.accu = []
        self.count = 0

    def one_pair(self, image, depth):
        image_batch = np.expand_dims(image, 0)
        out = self.p.compute(image_batch)
        out = np.squeeze(out['depth'])
        out = cv2.resize(out, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_LINEAR)

        out = self.baseline * self.focal / (out + 0.0000001)
        out = out / 800.0 # some magic number that I made up 
        global debug 
        debug = out

        errors = self._compute_errors(depth+1e-5, out+1e-5)
        self.accu.append(errors)
        self.count += 1

    def summarize(self):
        names = ['abs_rel', 'sq_rel', 'rmse', 'rmse_log', 'a1', 'a2', 'a3']
        errors = np.stack(self.accu, axis=0)
        errors = np.mean(errors, 0)
        out = {}
        for i in range(len(names)):
            out[names[i]] = errors[i]
        return out


modality = "depth" # "seg"

perf = {"depth": DepthPerformance, "seg":None}[modality]()
results = {}
#for town in ["Town01", "Town02"]:
for town in ["Town02"]:
    results[town] = {}
    for weather in range(1, 14):
        perf.reset()
        path = "/scratch/yang/aws_data/carla_collect/"+town + "_allsensor/default_RotationPitch=0_WeatherId=" +str(weather).zfill(2) + "/data_00000.h5"
        f = h5py.File(path, "r")
        for i in range(200):
            #print(i)
            image = f['CameraMiddle'][i]
            image = cv2.imdecode(image, 1)
            image = image[:,:, ::-1]

            if modality == "seg":
                seg = f['SegMiddle'][i]
                mode_image = cv2.imdecode(seg, 0)
            elif modality == "depth":
                mode_image = f['DepthMiddle'][i]
                mode_image = cv2.imdecode(mode_image, 1)
                # TODO: convert it
                def to_depth(array):
                    # input is bgr
                    array = array.astype(numpy.float32)
                    # Apply (R + G * 256 + B * 256 * 256) / (256 * 256 * 256 - 1).
                    normalized_depth = numpy.dot(array[:, :, :], [65536.0, 256.0, 1.0])
                    normalized_depth /= 16777215.0 / 1000.0  # (256.0 * 256.0 * 256.0 - 1.0)
                    return normalized_depth
                mode_image = to_depth(mode_image)
            else:
                raise
            
            perf.reset()
            perf.one_pair(image, mode_image)
            print(perf.summarize())
            raise
            break
        f.close()
        print(town, "weather", weather, perf.summarize())
        results[town][weather] = perf.summarize()
        with open(modality + ".pkl", "w") as ppp:
            pickle.dump(results, ppp)



In [ ]:
mode_image.dtype

In [ ]:
(mode_image[:,:,0]==mode_image[:,:,2]).all()

In [ ]:
np.histogram(mode_image)

In [ ]:
def to_gray(normalized_depth):
    normalized_depth = normalized_depth /1000.0
    logdepth = numpy.ones(normalized_depth.shape) + \
        (numpy.log(normalized_depth) / 5.70378)
    logdepth = numpy.clip(logdepth, 0.0, 1.0)
    logdepth *= 255.0
    logdepth = np.uint8(logdepth)
    # Expand to three colors.
    return numpy.repeat(logdepth[:, :, numpy.newaxis], 3, axis=2)

In [ ]:
d=to_gray(mode_image)

In [ ]:
d.dtype

In [ ]:
mode_image

In [ ]:
Image.fromarray(to_gray(mode_image))

In [ ]:
np.histogram(debug/3)

In [ ]:
84642.0 / mode_image.size 

In [ ]:
np.histogram(mode_image)

In [ ]:
Image.fromarray(to_gray(debug/3))

# covnert matthias data to my format


In [9]:
path = "/scratch/yang/aws_data/CIL_modular_data/matthias_data/RC28_wpz_M/data_00000.h5"
import h5py
f=h5py.File(path, "r")

In [10]:
f.keys()

[u'depth', u'labels', u'rgb', u'targets']

In [11]:
f['rgb'].shape

(200, 88, 200, 3)

In [3]:
out = "/scratch/yang/aws_data/carla_collect/matthias_converted/train/data_00000.h5"
hf=h5py.File(out, "w")

In [4]:
import numpy as np
data_rewards = hf.create_dataset('targets', (200, 35), 'f')
dt = h5py.special_dtype(vlen=np.dtype('uint8'))
sensor = hf.create_dataset("CameraMiddle", (200,), dtype=dt)

In [5]:
import cv2, math
for i in range(200):
    image = f['rgb'][i]
    image = cv2.imencode(".jpg", image)[1]
    encoded = np.fromstring(image, dtype=np.uint8)
    sensor[i] = encoded
    
    line = f["targets"][i]
    # change speed
    speed_kmh = line[10]
    line[10] /= 3.6
    # augment steer
    angle = math.radians(30.0)
    time_use = 1.0
    car_lenght = 6.0
    delta = min(6 * (math.atan((angle * car_lenght) / (time_use * speed_kmh + 0.05))) / math.pi, 0.3)
    line[0] -= np.sign(line[26])*delta
    
    data_rewards[i, :] = line

hf.close()
f.close()

/home/yang/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [6]:
np.sign(0.01)

1.0

In [8]:
f.keys()

ValueError: Not a location id (Invalid object id)

In [13]:
path = "/scratch/yang/aws_data/CIL_modular_data/_benchmarks_results/mm45_v4_base_newseg_YangExp_Town02/metrics.json"
import json
obj=json.load(open(path, "r"))

In [15]:
obj.keys()

[u'intersection_otherlane',
 u'intersection_offroad',
 u'collision_other',
 u'driven_kilometers',
 u'collision_pedestrians',
 u'average_speed',
 u'episodes_fully_completed',
 u'episodes_completion',
 u'collision_vehicles']

In [20]:
obj['episodes_fully_completed'].keys()
t = obj['episodes_fully_completed']

In [24]:
weathers = {"train": ["1.0", "10.0"],
           "val": ["13.0", "14.0"]}
for phase in ["train", "val"]:
    for weather in weathers[phase]:
        for task in range(len(t[weather])):
            perf = np.mean(t[weather][task])
            try:
                nsample = len(t[weather][task])
            except TypeError:
                nsample = 0
            
            print(phase, " weather ", weather, " task ", task, " average performance ", perf, "(num sample %d)" % nsample)


('train', ' weather ', '1.0', ' task ', 0, ' average performance ', 0.6, '(num sample 15)')
('train', ' weather ', '1.0', ' task ', 1, ' average performance ', 0.72, '(num sample 25)')
('train', ' weather ', '10.0', ' task ', 0, ' average performance ', 0.8666666666666667, '(num sample 15)')
('train', ' weather ', '10.0', ' task ', 1, ' average performance ', 0.88, '(num sample 25)')
('val', ' weather ', '13.0', ' task ', 0, ' average performance ', 0.7333333333333333, '(num sample 15)')
('val', ' weather ', '13.0', ' task ', 1, ' average performance ', 0.11764705882352941, '(num sample 17)')
('val', ' weather ', '14.0', ' task ', 0, ' average performance ', 0.0, '(num sample 0)')
('val', ' weather ', '14.0', ' task ', 1, ' average performance ', 0.8, '(num sample 15)')


In [5]:
import sys
sys.path.append("./drive_interfaces/carla/carla_client/carla/driving_benchmark/experiment_suites/")

import experiment_suite
class YangExp(experiment_suite.ExperimentSuite):

    def set_train_weathers(self, weathers):
        self.train_weathers_data = weathers

    @property
    def train_weathers(self):
        return self.train_weathers_data


In [6]:
a = YangExp("Town01")

In [7]:
a.set_train_weathers([1])

In [8]:
a.train_weathers

[1]

In [7]:
!cat ./drive_interfaces/carla/carla_client/carla/driving_benchmark/experiment_suites/experiment_suite.py

# To be redefined on subclasses on how to calculate timeout for an episode
import abc


class ExperimentSuite(object):

    def __init__(self, city_name):

        self._city_name = city_name
        self._experiments = self.build_experiments()

    def calculate_time_out(self, path_distance):
        """
        Function to return the timeout ,in milliseconds,
        that is calculated based on distance to goal.
        This is the same timeout as used on the CoRL paper.
        """
        return 8*(((path_distance / 1000.0) / 10.0) * 3600.0 + 500010.0)

    def get_number_of_poses_task(self):
        """
            Get the number of poses a task have for this benchmark
        """

        # Warning: assumes that all tasks have the same size

        return len(self._experiments[0].poses)

    def get_experiments(self):
        """
        Getter for the experiment set.
        """
        return self._experiments

    @property
    def dynamic_tasks(self):
        """
        Ret